# Model Project: The Agents Labour Supply Decision


In present project we seek to model the supply of labour by studying a two-good static problem, where agents value leisure and consumption. As an extension for the base model we impose a progressive tax system, which creates kinks in the budget constraint. We then show, how this may create bunching around tax bracket cut-offs.

The basic problem of the agents is:


$$\begin{aligned}  \max_{c,l}\ \ &u(c,l) \quad s.t. \ \ R_0 = wl + c  \quad c,l\geq0\\
 &\dfrac{\partial u}{\partial a}= u_a>0 \quad  \wedge \quad \dfrac{\partial^2 u}{\partial a^2}=u_{aa}<0,  \qquad a=c,l \\
 &R_0=wT + A_0 
 \end{aligned} $$
 

Where $R_0$ is the income potential of the agent, $c$ the consumption which is a numeraire, $l$ is the amount of leisure consumed with the associated oppurtunity cost of being the wage rate, $w$. We assume marginal utility of consumption and leisure are positive but decreasing. Further we for now assume the income potential is the sum of wage earnings from working your entire time endowment, $T$, and non-labour income, $A_0$. The problem is framed such that the agent at default suplies all his time as labour, giving her/him an income of $R_0$. The agent now has to decide how much leisure she/he will 'buy back', considering the consumption potential which is implied.

The Langrangian for the problem is:
$$ \mathcal{L}(c,l; \lambda)= u(c,l)\ + \ \lambda( R_0 - wl - c) $$ 
For which the first order conditions with regards to $c$ and $l$ implies an optimum at $\dfrac{u_l}{u_c}=w$ as the constraint is binding.

## Numerical optimization

We will solve this problem using a numerical optimizer, which will allow for easy handling of extensions to this basic model. 

Dependencies:

In [20]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy.optimize import Bounds
import scipy.integrate as integrate
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

First we code up two utility functions, a CES and a Cobb-Douglas, but any utility function that observes the above restrictions can be implemented.

In [21]:
# Cobb-Douglas:
def  _cobbdouglas(c,l):
    """ Cobb douglas utility function for 2 goods.
    PARAMETERS
    alpha: income share spent on consumption
    INPUT:
    c: consumption
    l: leisure

    OUTPUT:
    u: utility (negative)"""
    u= (c**alpha)*l**(1-alpha)
    return -u
def  cobbdouglas(x):
    return _cobbdouglas(c=x[0],l=x[1])

# CES utility function:
def _CES(c,l):
    """ CES utility function for 2 goods.
    PARAMETERS:
    r: 1/(1-r) is the elasticity of substitution, r<=1.
    a: relative preference for consumption (1 unit of consumption
     gives the same utility as 1 unit lesiure), 0<=a<=1.
    INPUT:
    c: consumption (dollars spent)
    l: leisure (hrs pr. week)

    OUTPUT:
    u: utility (neg) """
    u=(a*c**r + (1-a)*l**r)**(1/r)
    return -u
def CES(x):
    """ Takes a tuple as argument:
    ARGS
    x is a 2x1
    x0= consumption
    x1= leisure
    OUTPUT
    utility from consumption
    """
    return _CES(c=x[0],l=x[1]) 

We now have to define the constraints. Three moving parts are specified for the budget constraint (BC):<br>
1) Wage function: returning the after tax wagerate for for a given level of leisure consumption (the more leisure the lower taxrate = higher wagerate).<br>
2) Leisure expenditure function: returns the expenditure incurred from buying 'l' units of leisure. <br>
3) The budget function: calculates the slack in budget given choices of leisure and consumption (the actual constraint). <br>

In [29]:
# 1) Wage function: returning the after tax wagerate for for a given level of leisure consumption (the more leisure the lower taxrate = higher wagerate).<br>
def wage(l):
    """ Constant wage function:
        
        PARAMETERS:
        w: constant wage
        ARGS:
        l: leisure 
        
        OUTPUT:
        The marginal after tax wage rate
    """
    return w
#2) Leisure expenditure function:
def leiexp(l,wage):
    """ Calculates the expenditure from buying l units of leisure given a wage function as the definite integral
    from 0 to l of the wage function.
    ARGS:
    l: leisure consumed (int)
    Wage: Marginal wage-function (incoperating tax system) (function)
    
    OUTPUT:
    leiexp: expenditure on leisure (int)
    """
    return sp.integrate.quad(wage,0,l)[0]

#3) Budget constraint
def budget(x):
    """The budget functin calculates the slack for consumption plan:
    ARGS
    x: (2-tuple)
        x[0]: consumption of good
        x[1]: consumption of leisure
    
    Other inputs:
    A: non-labour income (int)
    wage(l): marg. wage/price of leisure (function)
    
    OUTPUT:
    slack: 'Leftover' given consumption plan x.
    """
    c=x[0]
    l=x[1]
    R0= maxlabinc + A
    slack= R0 - leiexp(l, wage) - c
    return  slack
budget_con={'type':'eq', 'fun':budget} # Impose equality contraint -> slack shall be zero in optimum

Note: 1) that we define the wage as a function this allows us to extend the model, 2) we have yet to define the bounds on consumption/leisure, as these depend on T, we will define them within when parameters are chosen.



## Example: Cobb Douglas utility
For a Cobb-Douglas utility functions, $u(c,l)= c^\alpha l^{1-\alpha}$, the solution Marshallian demand functions are:
$$c^*= w l \left(\dfrac{\alpha}{1-\alpha}\right)=R_0 \alpha \qquad l^*=\dfrac{R_0 (1-\alpha)}{w}$$
These can be obtained by isolating factor demand for one good from the optimum condition and inserting this into the budget constraint.
However to initially test our program, we will compare the analytical solution for a particular set of parameters to the optimizer:

In [31]:
# Budget related parameters
T=10 # 10 units of time endowed
w=10 # flat hourly wage of 10
A=20 # 20 income from non labour sources
maxlabinc=leiexp(T,wage)
# Utility related parameters
alpha=0.5 # Assuming

# Optimizer setup:
bounds= Bounds([0,np.inf], {0, T})

Given these parameters, we find $R_0=10\times10 + 20$, and analytical solution for optimal consumption choices are $c^*=120\times0.5=60$ and $l^*=\dfrac{120\times0.5}{10}=6$. <br>
The numerical optmization is performed by the 'SLQP' algorithm for constrained optimization included in sci.py:

In [32]:
guess=(5, 70) 
result = optimize.minimize(cobbdouglas,guess,
                             method='SLSQP',
                             constraints=[budget_con],
                             options={'disp':True})
 

print('\nx = ',result.x)


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -18.97366574611347
            Iterations: 10
            Function evaluations: 40
            Gradient evaluations: 10

x =  [59.99096963  6.00090304]


Deviations in the analytical and numerical solutions can be chalked up to numerical precision.